In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import pandas_profiling
import numpy as np
import re
import array as arr
import datetime

In [2]:
df = pd.read_excel('/Users/swatz_bachoo/Downloads/HMS/PROBLEM2/SGD-Features.xlsx', sheet_name = 'ORFs left arm of chr11')
print("Column headings:")
print(df.columns)

Column headings:
Index(['Row', 'Primary SGDID', 'Feature type', 'Feature name', 'Gene name',
       'Secondary SGDID', 'Chromosome', 'Start coordinate', 'Stop coordinate',
       'Strand', 'Sequence version date', 'Description', 'Unnamed: 12'],
      dtype='object')


In [3]:
"""Renaming columns
It is good practice to avoid starting column names in capitals or have white spaces. 
Uppercase has been converted to lowercase
Spaces have been replaced with underscore"""
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
print(df.columns)

Index(['row', 'primary_sgdid', 'feature_type', 'feature_name', 'gene_name',
       'secondary_sgdid', 'chromosome', 'start_coordinate', 'stop_coordinate',
       'strand', 'sequence_version_date', 'description', 'unnamed:_12'],
      dtype='object')


In [4]:
"""Data exploration: Provides a HTML report of the dataframe"""
print ("Exploratory report of the dataframe in .html format")
pandas_profiling.ProfileReport(df)

Exploratory report of the dataframe in .html format


Number of variables,13
Number of observations,84
Total Missing (%),13.3%
Total size in memory,8.6 KiB
Average record size in memory,105.0 B
Numeric,3
Categorical,8
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [5]:
"""No of rows*columns in df"""
print ("Rows*Columns in the dataframe")
df.shape

Rows*Columns in the dataframe


(84, 13)

In [6]:
print ("Head of the given dataframe")
df.head()

Head of the given dataframe


,row,primary_sgdid,feature_type,feature_name,gene_name,secondary_sgdid,chromosome,start_coordinate,stop_coordinate,strand,sequence_version_date,description,unnamed:_12
0,1,S000001568,ORF,YKL085W,MDH1,L000001045,11.0,279123,280127,W,1996-07-31,Mitochondrial malate dehydrogenase; catalyzes ...,NaN
1,2,S000001569,ORF,YKL086W,SRX1,NaN,11.0,278281,278664,W,1996-07-31,Sulfiredoxin; contributes to oxidative stress ...,NaN
2,7,S000001574,ORF,YKL091C,NaN,NaN,11.0,270650,269718,C,1996-07-31,Putative phosphatidylinositol/phosphatidylchol...,NaN
3,8,S000001575,ORF,YKL092C,BUD2,L000000199,11.0,269459,266145,C,1996-07-31,GTPase activating factor for Rsr1p/Bud1p; play...,NaN
4,9,S000001576,ORF,YKL093W,MBR1,L000001033,11.0,264789,265808,W,1996-07-31,protein involved in mitochondrial functions an...,NaN


In [7]:
"""Dropping the not needed "Unnamed: 12" column. 
Changes to be made directly in our object"""
to_drop = ['unnamed:_12']
df.drop(to_drop, inplace = True, axis = 1)
print("Wanted Columns in the dataframe:\n ",df.columns)

Wanted Columns in the dataframe:
  Index(['row', 'primary_sgdid', 'feature_type', 'feature_name', 'gene_name',
       'secondary_sgdid', 'chromosome', 'start_coordinate', 'stop_coordinate',
       'strand', 'sequence_version_date', 'description'],
      dtype='object')


In [8]:
"""Missing values in the mandatory Primary SGDID column"""
df.loc[(df['primary_sgdid'].isnull()), "row"]

71    114
Name: row, dtype: int64

In [9]:
"""Checking if the column Primary SGDID has unique values for each record"""
df['primary_sgdid'].is_unique


False

In [10]:
"""Setting Primary SGDID as the primary key"""
df = df.set_index('primary_sgdid')


In [11]:
"""Duplicate Records based on Primary SGDID unique identifier values"""
df_duplicates = pd.concat(g for _, g in df.groupby("primary_sgdid") if len(g) > 1)
df_duplicates['row']


primary_sgdid
S000001654    96
S000001654    97
Name: row, dtype: int64

In [12]:
"""Removing the duplicate records. 
Do not include index; Row is not the same. So check duplicates in a combo of other col values"""
df.drop_duplicates(subset=['feature_type','start_coordinate', 'stop_coordinate',
       'strand'], keep='first', inplace=True)
df.loc[df['row'] == 97]
print ("Duplicate records have been removed")
"""df.loc[df['row'] == 96] Row 97 record is dropped. Row 96 record now exists"""

Duplicate records have been removed


"df.loc[df['row'] == 96] Row 97 record is dropped. Row 96 record now exists"

In [13]:
"""Checking which columns have missing values"""
"""To check which values are missing, use
df.isna()"""
df.columns[df.isnull().any()]

Index(['feature_name', 'gene_name', 'secondary_sgdid', 'chromosome',
       'description'],
      dtype='object')

In [14]:
"""Computing number of missing values in each column"""

df.isna().sum()

row                       0
feature_type              0
feature_name              2
gene_name                20
secondary_sgdid          36
chromosome                1
start_coordinate          0
stop_coordinate           0
strand                    0
sequence_version_date     0
description               1
dtype: int64

In [15]:
"""
6 Mandatory columns : 
row, primary_sgdid, feature_type, start_coordinate, stop coordinate, and strand.
We can see from the above chunk that none of these mandatory columns have missing values.
However, if we have to locate which rows in these columns have missing values using other data sets, then 
the following command(S) can be used:
If you want only the row values in an array:
df.loc[(df['feature_type'].isnull()), "row"].values
If you want both the unique identifier and corresponding row values:
df.loc[(df['feature_type'].isnull()), "row"]
See example on an optional column with missing values below
The same can be checked on other manadatory columns"""
print("None of the other mandatory columns have missing values as seeen above.\nSample of how to obtain rows with missing values in the optional gene-name column")
df.loc[(df['gene_name'].isnull()), "row"].values


None of the other mandatory columns have missing values as seeen above.
Sample of how to obtain rows with missing values in the optional gene-name column


array([  7,  15,  18,  31,  35,  36,  39,  44,  52,  54,  57,  76,  79,
        85,  86,  90,  94, 147, 148, 150])

In [16]:
"""Examining datatypes of the dataframe columns"""
print("Examining datatypes of the dataframe columns")
df.dtypes

Examining datatypes of the dataframe columns


row                        int64
feature_type              object
feature_name              object
gene_name                 object
secondary_sgdid           object
chromosome               float64
start_coordinate           int64
stop_coordinate            int64
strand                    object
sequence_version_date     object
description               object
dtype: object

In [17]:
"""Rows where feature_type value is not ORF.
CDS is still an ORF containing only the coding regions of the gene"""
print("Row values where feature type is not an ORF or a CDS")
searchfor = ['ORF', 'CDS']
df.loc[((df['feature_type'].notnull()) & (~df['feature_type'].str.contains(('|'.join(searchfor)), na=False))), ('row', 'feature_type')] 


Row values where feature type is not an ORF or a CDS


,row,feature_type
primary_sgdid,,
S000001619,57,ONF


In [18]:
""" The first three letters define the host origin (Y, for yeast), 
the specific chromosome (A through P to denote chromosomes I through XVI), 
and whether the ORF lies to the left (L) or right (R) of the
centromere.
Based on this feature_name can only start with YKL"""
print("Row values where feature names do not start with YKL, which is indicative of the q arm of chr11 in Saccharomyces. \nFeature-name in the First exception needs chomping to confirm with dataset values")
df.loc[((df['feature_name'].notnull()) & (~df['feature_name'].str.contains('^YKL.*', na=False))), ('row', 'feature_name')] 


Row values where feature names do not start with YKL, which is indicative of the q arm of chr11 in Saccharomyces. 
Feature-name in the First exception needs chomping to confirm with dataset values


,row,feature_name
primary_sgdid,,
S000001604,42,YKL121W
S000001640,81,APE2
S000001729,151,YKR021W


In [19]:
"""gene_name: pattern of 3 letters and a number
Catches exceptions in gene_names
Leaves out missing values as it is not manadtory"""
print("Row values where the gene names are not in the specified pattern")
df.loc[((df['gene_name'].notnull()) & (~df['gene_name'].str.contains('^[A-Z]{3}\d{1}', na=False))), ('row', 'gene_name')] 


Row values where the gene names are not in the specified pattern


,row,gene_name
primary_sgdid,,
S000001617,55,2013-10-01 00:00:00
S000001621,59,MRPL31
S000001653,95,MRPL38


In [20]:
"""alternate identifiers for the sequence feature  (optional, multiples separated by |)
Catches exceptions in secondary_sgdid
Leaves out missing values as it is not mandatory"""
print("Row values where the secondary sgdid is not in the alphanumeric format of a single letter followed by 9 numbers")
df.loc[((df['secondary_sgdid'].notnull()) & (~df['secondary_sgdid'].str.contains('^[A-Z]{1}\d{9}', na=False))), ('row', 'secondary_sgdid')] 


Row values where the secondary sgdid is not in the alphanumeric format of a single letter followed by 9 numbers


,row,secondary_sgdid
primary_sgdid,,
S000001582,17,chromosome 11


In [21]:
"""It is mentioned that the ORFs are from Chromosome 11"""
print("Row values where the ORFs are not from Chr11")
df.loc[((df['chromosome'].notnull()) & (df['chromosome'] != 11)), ('row', 'chromosome')] 

Row values where the ORFs are not from Chr11


,row,chromosome
primary_sgdid,,
S000001695,137,13.0


In [22]:
"""Making sure that all dates are in the year-month-date format"""
df['sequence_version_date'] = pd.to_datetime(df['sequence_version_date']).apply(lambda x:x.strftime('%Y/%m/%d'))
print("Formatted Dates in the Data frame")
df['sequence_version_date']

Formatted Dates in the Data frame


primary_sgdid
S000001568    1996/07/31
S000001569    1996/07/31
S000001574    1996/07/31
S000001575    1996/07/31
S000001576    1996/07/31
S000001577    1996/07/31
S000001580    1996/07/31
S000001581    1996/07/31
S000001582    2011/02/03
S000001583    1996/07/31
S000001591    1996/07/31
S000001592    1996/07/31
S000001593    1996/07/31
S000001594    1996/07/31
S000001595    1996/07/31
S000001596    1996/07/31
S000001597    1996/07/31
S000033938    1996/07/31
S000001598    1996/07/31
S000001599    1996/07/31
S000001600    1996/07/31
S000001601    1996/07/31
S000001602    1996/07/31
S000001603    1996/07/31
S000001604    1996/07/31
S000001605    1996/07/31
S000001606    1996/07/31
S000001607    1996/07/31
S000001608    1996/07/31
S000001609    1996/07/31
                 ...    
S000001644    1996/07/31
S000001645    1996/07/31
S000007244    1999/07/17
S000001646    1996/07/31
S000001647    1996/07/31
S000001648    1996/07/31
S000007617    2001/02/26
S000001649    1996/07/31
S000001650 

In [23]:
"""strand values should be either W or C"""
strand_name = ['W', 'C']
print ("Row values where the strand value is not W or C")
df.loc[((df['strand'].notnull()) & (~df['strand'].str.contains(('|'.join(strand_name)), na=False))), ('row', 'strand')] 


Row values where the strand value is not W or C


,row,strand
primary_sgdid,,
S000001582,17,255720


In [24]:
"""Optional: last letter of feature_name denotes the strand
Can be used to impute strand values if they are missing"""
feature_name_values = df.loc[((df['feature_name'].notnull()) & (df['feature_name'].str.contains('^YKL.*', na=False))), ('feature_name')].values
last_letter = [x.strip()[-1] for x in feature_name_values]
print("Last letter of feature_name in the first five records: ",last_letter[0:5])



Last letter of feature_name in the first five records:  ['W', 'W', 'C', 'C', 'W']


In [25]:
"""Rows where start coordinate is lesser than stop coordinate"""
print("Rows where start coordinate is lesser than stop coordinate",df.loc[(df['start_coordinate'] < df['stop_coordinate']), ('row')])


Rows where start coordinate is lesser than stop coordinate primary_sgdid
S000001568      1
S000001569      2
S000001576      9
S000001577     10
S000001581     16
S000001582     17
S000001591     28
S000001592     29
S000001595     32
S000001600     38
S000001601     39
S000001603     41
S000001604     42
S000001606     44
S000001607     45
S000001608     46
S000001609     47
S000001610     48
S000001614     52
S000001619     57
S000001620     58
S000001628     67
S000001636     76
S000001637     77
S000001639     80
S000001640     81
S000001643     83
S000001646     87
S000001653     95
S000001654     96
S000001655     98
S000001667    111
S000001668    112
S000001693    135
S000001695    137
S000001706    148
S000001708    150
S000001729    151
Name: row, dtype: int64


In [26]:
"""The following row has mismatched column values"""
print("Row with mismatched column values: ",(df.iloc[8,0]))

Row with mismatched column values:  17


In [27]:
"""Sample demo of data drop
Obtaining the rows to be dropped based on the exceptions caught in the feature_name and feature_type cols"""
rows_feature_name = df.loc[((df['feature_name'].notnull()) & (~df['feature_name'].str.contains('^YKL.*', na=False))), ('row')].values
rows_feature_type = df.loc[((df['feature_type'].notnull()) & (~df['feature_type'].str.contains(('|'.join(searchfor)), na=False))), ('row')].values
rows_toDrop = np.concatenate((rows_feature_name, rows_feature_type))
_, i = np.unique(rows_toDrop, return_index=True)
rows_toDrop = rows_toDrop[np.sort(i)]
print("Cleaned Dataframe: \n",df[~df['row'].isin(rows_toDrop)])




Cleaned Dataframe: 
                row feature_type feature_name gene_name  \
primary_sgdid                                            
S000001568       1          ORF      YKL085W      MDH1   
S000001569       2          ORF      YKL086W      SRX1   
S000001574       7          ORF      YKL091C       NaN   
S000001575       8          ORF      YKL092C      BUD2   
S000001576       9          ORF      YKL093W      MBR1   
S000001577      10          ORF      YKL094W      YJU3   
S000001580      15          ORF      YKL097C       NaN   
S000001581      16          ORF      YKL098W      MTC2   
S000001582      17          ORF          NaN   YKL099C   
S000001583      18          ORF      YKL100C       NaN   
S000001591      28          ORF      YKL108W      SLD2   
S000001592      29          ORF      YKL109W      HAP4   
S000001593      30          ORF      YKL110C     KTI12   
S000001594      31          ORF      YKL111C       NaN   
S000001595      32          ORF      YKL112W      A

In [28]:
data_loss = (len(rows_toDrop)/ (df.shape [0] +1))*100
print("Expected data loss :",data_loss, "percent")

Expected data loss : 4.761904761904762 percent
